In [1]:
pip install tensorflow==1.15

     |████████████████████████████████| 412.3MB 42kB/s 
     |████████████████████████████████| 512kB 50.5MB/s 
     |████████████████████████████████| 3.8MB 38.7MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=3804118d75440fc55198250cd2284eead2c5c522c195c343c0a47f5064dfac6d
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.0
    Uninstalling tensorboard-2.4.0:
      Successfully uninstalled tensorboard-2.4.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully 

In [2]:
pip install matplotlib tqdm

In [3]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

In [4]:
from tqdm import tqdm_notebook, tqdm
from pandas.util.testing import *
from collections import deque

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  


In [5]:
class AI_Trader():

  def __init__(self, state_size, action_space=3, model_name='AITrader'):
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen = 2000)
    self.model_name = model_name
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    self.model = self.model_builder()
  
  def model_builder(self):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=32,
                                    activation = "relu",
                                    input_dim = self.state_size))
    model.add(tf.keras.layers.Dense(units=64,
                                    activation = "relu"))
    model.add(tf.keras.layers.Dense(units=128,
                                    activation = "relu"))
    model.add(tf.keras.layers.Dense(units=self.action_space,
                                    activation = "linear"))
    model.compile(loss = "mse",
                  optimizer = tf.keras.optimizers.Adam(lr=0.001))
    return model
  
  def trade(self, state):
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    actions = self.model.predict(state)
    return np.argmax(actions[0])

  def batch_train(self, batch_size):
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
    for state, action, reward, next_state, done in batch:
      if not done:
        reward = reward+self.gamma*np.amax(self.model.predict(next_state)[0])
      target = self.model.predict(state)
      target[0][action] = reward
      self.model.fit(state, target, epochs=1, verbose=0)
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

In [6]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [7]:
def stocks_price_format(n):
  if n < 0:
    return "- R$ {0:2f}".format(abs(n))
  else:
    return "R$ {0:2f}".format(abs(n))

In [8]:
dataset = stocks_price_format(100)
dataset = data_reader.DataReader("AAPL", data_source="yahoo")

In [9]:
print(dataset.head())
print(str(dataset.index[0]).split()[0])
print(dataset.index[-1])
print(dataset['Close'])

                 High        Low       Open      Close       Volume  Adj Close
Date                                                                          
2015-12-31  26.757500  26.205000  26.752501  26.315001  163649200.0  24.380093
2016-01-04  26.342501  25.500000  25.652500  26.337500  270597600.0  24.400942
2016-01-05  26.462500  25.602501  26.437500  25.677500  223164000.0  23.789471
2016-01-06  25.592501  24.967501  25.139999  25.174999  273829600.0  23.323915
2016-01-07  25.032499  24.107500  24.670000  24.112499  324377600.0  22.339539
2015-12-31
2020-12-28 00:00:00
Date
2015-12-31     26.315001
2016-01-04     26.337500
2016-01-05     25.677500
2016-01-06     25.174999
2016-01-07     24.112499
                 ...    
2020-12-21    128.229996
2020-12-22    131.880005
2020-12-23    130.960007
2020-12-24    131.970001
2020-12-28    136.690002
Name: Close, Length: 1257, dtype: float64


In [10]:
def dataset_loader(stock_name):
  dataset = data_reader.DataReader(stock_name, data_source="yahoo")
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  close = dataset['Close']
  return close

In [11]:
def state_creator(data, timestep, window_size):
  starting_id = timestep - window_size + 1

  if starting_id >= 0:
    windowed_data = data[starting_id:timestep + 1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep + 1])
  
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i + 1]))
  
  return np.array([state]), windowed_data

In [12]:
stock_name = "AAPL"
data = dataset_loader(stock_name)
s = state_creator(data, 0, 5)
window_size = 10
episodes = 1000
batch_size = 32
data_samples = len(data) - 1

In [13]:
trader = AI_Trader(window_size)

print(trader.model.summary())

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
for episode in range(1, episodes+1):
  print(f'Etapa: {episode} de {episodes}')
  state = state_creator(data, 0, window_size + 1)
  total_profit = 0
  trader.inventory = []
  for t in tqdm(range(data_samples)):
    action = trader.trade(state)
    next_state = state_creator(data, t + 1, window_size + 1)
    reward = 0

    if action == 1:
      trader.inventory.append(data[t])
      print("AI Trader comprou: ", stocks_price_format(data[t]))
    elif action == 2 and len(trader.inventory) > 0:
      buy_price = trader.inventory.pop(0)

      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader vendeu: ", stocks_price_format(data[t]),
            "Lucro de: " + stocks_price_format(data[t] - buy_price))
    
    if t == data_samples - 1:
      done = True
    else:
      done = False
    
    trader.memory.append((state, action, reward, next_state, done))
    state = next_state

    if done:
      print("#"*30)
      print(f'Lucro Total Estimado: {total_profit}')
      print("#"*30)

    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
    
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))

  0%|          | 0/1256 [00:00<?, ?it/s]

Etapa: 1 de 1000
AI Trader comprou:  R$ 26.315001
AI Trader vendeu:  R$ 26.337500 Lucro de: R$ 0.022499
AI Trader comprou:  R$ 24.112499
AI Trader comprou:  R$ 24.240000
AI Trader comprou:  R$ 24.632500
AI Trader comprou:  R$ 24.990000
AI Trader comprou:  R$ 24.347500
AI Trader comprou:  R$ 24.165001
AI Trader vendeu:  R$ 24.197500 Lucro de: R$ 0.085001
AI Trader vendeu:  R$ 24.075001 Lucro de: - R$ 0.164999
AI Trader comprou:  R$ 25.355000
AI Trader comprou:  R$ 24.860001
AI Trader comprou:  R$ 24.997499
AI Trader comprou:  R$ 23.355000
AI Trader vendeu:  R$ 24.334999 Lucro de: - R$ 0.297501
AI Trader vendeu:  R$ 24.107500 Lucro de: - R$ 0.882500
AI Trader vendeu:  R$ 23.620001 Lucro de: - R$ 0.727499
AI Trader vendeu:  R$ 24.087500 Lucro de: - R$ 0.077501
AI Trader comprou:  R$ 23.752501
AI Trader vendeu:  R$ 23.747499 Lucro de: - R$ 1.607500
AI Trader comprou:  R$ 23.567499
AI Trader comprou:  R$ 23.497499
AI Trader vendeu:  R$ 24.065001 Lucro de: - R$ 0.795000


  3%|▎         | 33/1256 [00:00<00:23, 52.29it/s]

AI Trader vendeu:  R$ 24.010000 Lucro de: - R$ 0.987499
AI Trader comprou:  R$ 24.219999


  3%|▎         | 35/1256 [00:00<01:11, 17.02it/s]

...

 85%|████████▍ | 1064/1256 [02:36<00:28,  6.74it/s]

AI Trader vendeu:  R$ 61.720001 Lucro de: R$ 1.167500
AI Trader vendeu:  R$ 61.380001 Lucro de: - R$ 1.834999


 85%|████████▍ | 1066/1256 [02:37<00:28,  6.74it/s]

AI Trader vendeu:  R$ 64.610001 Lucro de: R$ 7.299999
AI Trader vendeu:  R$ 61.935001 Lucro de: R$ 5.842503


 85%|████████▌ | 1068/1256 [02:37<00:27,  6.78it/s]